In [0]:
# https://keras.io/
!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 128
num_classes = 10
#epochs = 300
epochs = 50
l = 10
num_filter = 20

Using TensorFlow backend.


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 178s 1us/step


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=2)

In [0]:
def space_to_depth_x8(x):
  return tf.space_to_depth(x, block_size=8)

In [0]:
x_train[0].shape

(32, 32, 3)

In [0]:
''' 

input = Input(shape=(img_height, img_width, channel,))

# Layer 1
#layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)
#layer1 = BatchNormalization(name='norm_1')(layer1)
#layer1 = LeakyReLU(alpha=0.1)(layer1)

layer1 = LayerBlock(input)
layer1 = MaxPooling2D(pool_size=(2, 2))(layer1)

# Layer 2
layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)
layer2 = BatchNormalization(name='norm_2')(layer2)
layer2 = LeakyReLU(alpha=0.1)(layer2)
layer2 = MaxPooling2D(pool_size=(2, 2))(layer2)

# Layer 3
layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer2)
layer3 = BatchNormalization(name='norm_3')(layer3)
layer3 = LeakyReLU(alpha=0.1)(layer3)

# Layer 4
layer4 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer3)
layer4 = BatchNormalization(name='norm_4')(layer4)
layer4 = LeakyReLU(alpha=0.1)(layer4)

# Layer 5
layer5 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_5', use_bias=False)(layer4)
layer5 = BatchNormalization(name='norm_5')(layer5)
layer5 = LeakyReLU(alpha=0.1)(layer5)
layer5 = MaxPooling2D(pool_size=(2, 2))(layer5)

# Layer 6
layer6 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_6', use_bias=False)(layer5)
layer6 = BatchNormalization(name='norm_6')(layer6)
layer6 = LeakyReLU(alpha=0.1)(layer6)

# Layer 7
layer7 = Conv2D(128, (1,1), strides=(1,1), padding='same', name='conv_7', use_bias=False)(layer6)
layer7 = BatchNormalization(name='norm_7')(layer7)
layer7 = LeakyReLU(alpha=0.1)(layer7)

# Layer 8
layer8 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_8', use_bias=False)(layer7)
layer8 = BatchNormalization(name='norm_8')(layer8)
layer8 = LeakyReLU(alpha=0.1)(layer8)
layer8 = MaxPooling2D(pool_size=(2, 2))(layer8)

# Layer 9
layer9 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_9', use_bias=False)(layer8)
layer9 = BatchNormalization(name='norm_9')(layer9)
layer9 = LeakyReLU(alpha=0.1)(layer9)

# Layer 10
layer10 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_10', use_bias=False)(layer9)
layer10 = BatchNormalization(name='norm_10')(layer10)
layer10 = LeakyReLU(alpha=0.1)(layer10)

# Layer 11
layer11 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_11', use_bias=False)(layer10)
layer11 = BatchNormalization(name='norm_11')(layer11)
layer11 = LeakyReLU(alpha=0.1)(layer11)

# Layer 12
layer12 = Conv2D(256, (1,1), strides=(1,1), padding='same', name='conv_12', use_bias=False)(layer11)
layer12 = BatchNormalization(name='norm_12')(layer12)
layer12 = LeakyReLU(alpha=0.1)(layer12)

# Layer 13
layer13 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_13', use_bias=False)(layer12)
layer13 = BatchNormalization(name='norm_13')(layer13)
layer13_act = LeakyReLU(alpha=0.1)(layer13)

#skip_connection = layer13_act
skip_connection  = layer12

layer13 = MaxPooling2D(pool_size=(2, 2))(layer13_act)

# Layer 14
layer14 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_14', use_bias=False)(layer13)
layer14 = BatchNormalization(name='norm_14')(layer14)
layer14 = LeakyReLU(alpha=0.1)(layer14)

# Layer 15
layer15 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_15', use_bias=False)(layer14)
layer15 = BatchNormalization(name='norm_15')(layer15)
layer15 = LeakyReLU(alpha=0.1)(layer15)

# Layer 16
layer16 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_16', use_bias=False)(layer15)
layer16 = BatchNormalization(name='norm_16')(layer16)
layer16 = LeakyReLU(alpha=0.1)(layer16)

# Layer 17
layer17 = Conv2D(512, (1,1), strides=(1,1), padding='same', name='conv_17', use_bias=False)(layer16)
layer17 = BatchNormalization(name='norm_17')(layer17)
layer17 = LeakyReLU(alpha=0.1)(layer17)

# Layer 18
layer18 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_18', use_bias=False)(layer17)
layer18 = BatchNormalization(name='norm_18')(layer18)
layer18 = LeakyReLU(alpha=0.1)(layer18)

# Layer 19
layer19 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_19', use_bias=False)(layer18)
layer19 = BatchNormalization(name='norm_19')(layer19)
layer19 = LeakyReLU(alpha=0.1)(layer19)

# Layer 20
layer20 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_20', use_bias=False)(layer19)
layer20 = BatchNormalization(name='norm_20')(layer20)
layer20 = LeakyReLU(alpha=0.1)(layer20)

# Layer 21
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_21', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_21')(skip_connection)
skip_connection = LeakyReLU(alpha=0.1)(skip_connection)
skip_connection = Lambda(space_to_depth_x2)(skip_connection)

layer21 = concatenate([skip_connection, layer20])

# Layer 22
layer22 = Conv2D(1024, (3,3), strides=(1,1), padding='same', name='conv_22', use_bias=False)(layer21)
layer22 = BatchNormalization(name='norm_22')(layer22)
layer22 = LeakyReLU(alpha=0.1)(layer22)

# Layer 23
layer23 = Flatten()(layer22)

output = Dense(num_classes, activation='softmax')(layer23)

'''


" \n\ninput = Input(shape=(img_height, img_width, channel,))\n\n# Layer 1\n#layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_1', use_bias=False)(input)\n#layer1 = BatchNormalization(name='norm_1')(layer1)\n#layer1 = LeakyReLU(alpha=0.1)(layer1)\n\nlayer1 = LayerBlock(input)\nlayer1 = MaxPooling2D(pool_size=(2, 2))(layer1)\n\n# Layer 2\nlayer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_2', use_bias=False)(layer1)\nlayer2 = BatchNormalization(name='norm_2')(layer2)\nlayer2 = LeakyReLU(alpha=0.1)(layer2)\nlayer2 = MaxPooling2D(pool_size=(2, 2))(layer2)\n\n# Layer 3\nlayer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_3', use_bias=False)(layer2)\nlayer3 = BatchNormalization(name='norm_3')(layer3)\nlayer3 = LeakyReLU(alpha=0.1)(layer3)\n\n# Layer 4\nlayer4 = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_4', use_bias=False)(layer3)\nlayer4 = BatchNormalization(name='norm_4')(layer4)\nlayer4 = LeakyReLU(alpha=0.1)(layer4)\

In [0]:
def LayerBlock(layer_in, block_name):
  
  conv_prefix = block_name + '_conv_'
  norm_prefix = block_name + '_norm_'
  
  # layer1
  layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name=conv_prefix+'1', use_bias=False)(layer_in)
  layer1 = BatchNormalization(name=norm_prefix+'1')(layer1)
  layer1 = LeakyReLU(alpha=0.1)(layer1)

  # Layer 2
  layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name=conv_prefix+'2', use_bias=False)(layer1)
  layer2 = BatchNormalization(name=norm_prefix+'2')(layer2)
  layer2 = LeakyReLU(alpha=0.1)(layer2)
  
  # Layer 3
  layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name=conv_prefix+'3', use_bias=False)(layer2)
  layer3 = BatchNormalization(name=norm_prefix+'3')(layer3)
  layer3 = LeakyReLU(alpha=0.1)(layer3)
  
  # Layer 4
  layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name=conv_prefix+'4', use_bias=False)(layer3)
  layer4 = BatchNormalization(name=norm_prefix+'4')(layer4)
  layer4 = LeakyReLU(alpha=0.1)(layer4)

  # Layer 5
  layer5 = Conv2D(512, (3,3), strides=(1,1), padding='same', name=conv_prefix+'5', use_bias=False)(layer4)
  layer5 = BatchNormalization(name=norm_prefix+'5')(layer5)
  layer5 = LeakyReLU(alpha=0.1)(layer5)
  
  # MaxPooling
  out_layer = MaxPooling2D(pool_size=(2, 2))(layer5)
  
  return out_layer

In [0]:
input = Input(shape=(img_height, img_width, channel,))

block1 = LayerBlock(input, 'A')

print('block1:')
print(block1.shape)

block2 = LayerBlock(block1, 'B')

block3 = LayerBlock(block2, 'C')

block4 = LayerBlock(block3, 'D')

print('block4')
print(block4.shape)

####### concat block1 & block4
## reshape block1
block1_reshape = Lambda(space_to_depth_x8)(block1)

concat_layer = concatenate([block1_reshape, block4])

print('concat_layer')
print(concat_layer.shape)

############
## connect the concat layer to softmax thru conv layer
#########
layer_final = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_final', use_bias=False)(concat_layer)
layer_final = BatchNormalization(name='norm_final')(layer_final)
layer_final = LeakyReLU(alpha=0.1)(layer_final)
print('layer_final')
print(layer_final.shape)

##layer_final = Conv2D(10, (1,1), name='conv_final', use_bias=False)(concat_layer)

layer_end   = Conv2D(10, (2,2))(layer_final)

flatten_layer = Flatten()(layer_end)
output = Activation('softmax')(flatten_layer)


Instructions for updating:
Colocations handled automatically by placer.
block1:
(?, 16, 16, 512)
block4
(?, 2, 2, 512)
concat_layer
(?, 2, 2, 33280)
layer_final
(?, 2, 2, 32)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
A_conv_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
A_norm_1 (BatchNormalization)   (None, 32, 32, 32)   128         A_conv_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 32, 32, 32)   0           A_norm_1[0][0]                   
__________________________________________________________________________________________________
A_conv_2 (

In [0]:
!pip install graphviz

In [0]:
from keras.utils import plot_model

In [0]:
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
plot_model(model, show_shapes=True, to_file='model2.png')
#plot_model(model)

In [0]:
from google.colab import files

files.download('model2.png')

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
## to save the best model 
from keras.callbacks import ModelCheckpoint

In [0]:
?ModelCheckpoint

In [0]:
checkpoint = ModelCheckpoint('3C_best_model.hdf5', monitor='val_acc', verbose=1, save_best_only=True)

callbacks = [checkpoint]

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 288s 6ms/step - loss: 1.2898 - acc: 0.5425 - val_loss: 2.0136 - val_acc: 0.4564

Epoch 00001: val_acc improved from -inf to 0.45640, saving model to 3C_best_model.hdf5
Epoch 2/50
50000/50000 [==============================] - 272s 5ms/step - loss: 0.8088 - acc: 0.7166 - val_loss: 1.0581 - val_acc: 0.6498

Epoch 00002: val_acc improved from 0.45640 to 0.64980, saving model to 3C_best_model.hdf5
Epoch 3/50
50000/50000 [==============================] - 271s 5ms/step - loss: 0.6366 - acc: 0.7790 - val_loss: 1.0978 - val_acc: 0.6473

Epoch 00003: val_acc did not improve from 0.64980
Epoch 4/50
50000/50000 [==============================] - 271s 5ms/step - loss: 0.5229 - acc: 0.8168 - val_loss: 0.7821 - val_acc: 0.7382

Epoch 00004: val_acc improved from 0.64980 to 0.73820, saving model to 3C_best_model.hdf5
Epoch 5/50
50000/50000 [=====

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")

10000/10000 [==============================] - 23s 2ms/step
Test loss: 1.1042507146820426
Test accuracy: 0.8264
Saved the model to disk


In [0]:
!pwd


/content


In [0]:
model.save_weights("Yolo_Basic_model2.h5")
print("Saved the model to disk")
from google.colab import files

#files.download('/content/Yolo_Basic_model2.h5')